In [ ]:
import numpy as np
from numpy.typing import NDArray
from numpy import random

Variables modèle

In [ ]:
# Nombre de tâches à effectuer
N : int
# Nombre de serveurs MEC
M : int

# Taille max de l'archive HxWxL
REFPOP : int


# MEC parent de la tâche n (entre 1 et M)
MEC = np.zeros(N,dtype=int)

Variables solutions

In [ ]:
# 1 si la tâche n est exécutée sur le serveur m.
X = np.zeros((N,M),dtype=int)

# capacité maximale du serveur m (en unités de VM)
V = np.zeros(M,dtype=int)

# taille de la tâche n (Mbits)
G = np.zeros(N,dtype=int)
# taille du resultat de la tâche n (Mbits)
RG = np.zeros(N,dtype=int)

# Variables de décision
# MEC_exe(n)
# Note : if MEC(n) = -1 then it is treated with cloud
MEC_exe = np.array([-1 for _ in range(N)])
# besoin VM de la tâche n
U = np.zeros(N,dtype=int)

# j=0 local MEC, j=1 other MEC, j=2 cloud
J = np.array([2 for _ in range(N)])

# le nombre de VM allouées au tâches effectuées sur un MEC (non cloud) 
def U_NC_func(U,MEC_exe):
    return U * np.heaviside(MEC_exe,0)

Constantes du réseau

In [ ]:
# Bandes passantes en Mb/s
BP_p : float = 1 # local
BP_q : float = 1 # inter-MEC
BP_r : float = 1 # MEC-cloud

# La performance unitaire d’une VM en Mb/s
P_u : float = 1

# Energie
# Puissances électrique en W
r_v : float = 1 #puissance d'un serveur à vide
r_o : float = 1 #puissance d'une VM occupée
r_u : float = 1 #puissance d'une VM innocupée

Coût Temporel - F1


In [ ]:
# Util
BP_cumsum = np.cumsum(np.array([1/BP_p,1/BP_q,1/BP_r]))

# Temps de transmission de la requête
# dimension 3*N
def T1_func(G,BP=BP_cumsum):
    return np.outer(G, BP)

# Temps de transmission du résultat
def T3_func(RG,BP=BP_cumsum):
    return np.outer(RG, BP)

# Temps de calcul du résultat
def T2_func(G,U,P_u=P_u):
    return G / (U * P_u)

# Temps total global
def Ttot_func(G,RG,U, J, N=N):
    """
    Returns the total time i.e. the duration of the longest task to perform.

    PARAMETERS:
    
      G : array of tasks size.
      RG : array of tasks result size.
      U : array of tasks VM allocated.
      J : array of tasks location in the network, j=0 performed on local MEC, j=1 other MEC, j=2 cloud.

    RETURNS:
      int: total_time
    """
    T1, T2, T3=T1_func(G), T2_func(G,U), T3_func(RG)
    T = np.zeros(N)
    for n,j in enumerate(J):
        T[n] = (T1[n][j] + T2[n][j] + T3[n][j])
    return max(T)

Consommation énergétique - F2

In [ ]:
# Coût énergétique à vide des serveurs en kW.h
def EC_v_func(Ttot,M=M, r=r_v):
    return M*r*Ttot

# Coût énergétique lié aux VM allouées
def EC_o_func(U_NC,T2,r=r_o):
    return r*sum(U_NC*T2)

# Coût énergétique lié aux VM non allouées
def EC_u_func(U_NC, T2, Ttot, V=V, r=r_u):
    return r * (sum(V*Ttot) - sum(U_NC*T2))

# Consommation totale
def ECtot_func(T2, Ttot,U_NC):
    EC = EC_v_func(Ttot)+EC_o_func(U_NC,T2)+EC_u_func(T2,Ttot,U_NC)
    return EC

Répartition des tâches - F3

In [ ]:
# Equilibrage des charges
def MSU_func(U: NDArray,X: NDArray):
    """
    Calcul pour l'équilibrage des charges (Mean Square Unbalance).

    Parameters  
      U : (N,) vector.
      X : (N,M) matrix.
    """
    Nu = U@X
    return np.var(Nu)

Chromosome

In [ ]:
def C_func(MEC_exe,U):
    Chromosome = np.stack((MEC_exe,U), axis=1)
    return Chromosome

In [ ]:
# Util
def appsOfServ_m(MEC_exe:NDArray,m:int):
    appsOfServ_m = []
    for n in range(len(MEC_exe)):
        if MEC_exe[n] == m:
            appsOfServ_m.append(n)
    return appsOfServ_m

Evaluate Solution

In [ ]:
def evaluateSolution(chromosome:NDArray, n:int, V=V):
    """
    Returns True if chromosome is a valide solution.
    Parameters:
      chromosome : solution to evaluate.
      n : last changed app to specifically check on.
    """
    MEC_exe_chr, U_chr = chromosome.T
    # We check for the server MEC recently changed
    m = MEC_exe_chr[n]

    # Variable domain constraint
    if U_chr[n] <= 0:
        return False

    appsOfServ_m = appsOfServ_m(MEC_exe_chr,m)
    # Server capacity constraint
    U_for_m = 0
    for app in appsOfServ_m:
        U_for_m += U_chr[app]
    if U_for_m > V[m]:
        return False    

    # if valide solution returns True
    return True

Generate Solution

In [ ]:
def mutate(chromosome: NDArray, X: NDArray, J:NDArray, random_split=0.5, N=N, M=M, MEC=MEC):
    """
    ! This Version does not include Cloud yet. !

    Apply mutations : change of MEC affectation, and adding/removing VM capacity
    Only one type of mutation occurs at each call, based on the random_split. If less than (ramdom_split*100)% MEC 
    affectation mutation occurs, else VM capacity mutation occurs.

    Parameters:
      chromosome : Current chromosome to mutate.
      X : Adapt the mutant X matrix to the mutation.
      random_split (default 0.5) : defines the split for each mutation to occur.
      N (default global variable N) : number of tasks.
      M (default global variable M) : number of MEC servers.
    """
    
    stopLoop = False

    # Generate mutation until a valide solution comes out
    while stopLoop is not True:

      mutant = chromosome.copy()
      mutant_MEC_exe, mutant_U = mutant.T

      mutant_X = X.copy()
      mutant_J = J.copy()

      active_n = random.randint(N)
      newMec = random.randint(M)
      
      random_step = random.randint(100)

      # Randomly change a MEC affectation
      if random_step < random_split*100:
         oldMec = mutant_MEC_exe[active_n]
         mutant_MEC_exe[active_n] = newMec

         # Apply changes on X
         mutant_X[active_n][oldMec] = 0
         mutant_X[active_n][newMec] = 1

         # Apply changes on J
         if MEC[active_n]==newMec:
             mutant_J[active_n] = 0
         else:
             mutant_J[active_n] = 1

      # Randomly add or substract a VM allocation
      else:
         # Choose between adding VM or removing VM capacity with 50% chance
         if random.randint(2) >= 1:
               mutant_U[active_n] += 1
         else:
               mutant_U[active_n] -= 1

      stopLoop = evaluateSolution(mutant,active_n)


    return mutant, mutant_X, mutant_J

PAES Step

In [ ]:
def paes_step(currentSol_candidateSol:NDArray, current_X: NDArray, current_J: NDArray, candidate_X: NDArray, candidate_J: NDArray, archive:NDArray, G=G, RG=RG, refpop=REFPOP):
    currentSolution, candidateSolution = currentSol_candidateSol.T
    
    # Current Solution
    cu_MEC_exe, cu_U = currentSolution.T
    cu_U_NC = U_NC_func(cu_U,cu_MEC_exe)
    cu_T2 = T2_func(G,cu_U)

    # Objective functions for current solution
    cu_Ttot = Ttot_func(G,RG,cu_U,current_J)
    cu_ECtot = ECtot_func(cu_T2,cu_Ttot,cu_U_NC)
    cu_MSU = MSU_func(cu_U,current_X)

    # Candidate Solution
    ca_MEC_exe, ca_U = candidateSolution.T
    ca_U_NC = U_NC_func(ca_U,ca_MEC_exe)
    ca_T2 = T2_func(G,ca_U)

    # Objective functions for candidate solution
    ca_Ttot = Ttot_func(G,RG,ca_U,candidate_J)
    ca_ECtot = ECtot_func(ca_T2,ca_Ttot,ca_U_NC)
    ca_MSU = MSU_func(ca_U,candidate_X)

    # Current dominates Candidate
    if cu_Ttot <= ca_Ttot and cu_ECtot <= ca_ECtot and cu_MSU <= ca_MSU:
        # Nothing to change
        return currentSol_candidateSol
    
    # Candidate dominates Current
    elif cu_Ttot > ca_Ttot and cu_ECtot > ca_ECtot and cu_MSU > ca_MSU:
        # Candidate replaces Current
        currentSolution = candidateSolution.copy()
        return currentSol_candidateSol
    
    else:
        dominated_solutions = []

        for id,NDS_solution in enumerate(archive.flat):
            # L'archive stocke la solution et les valeurs des fonctions objectives correspondantes
            tmp_solution, tmp_Ttot, tmp_ECTot, tmp_MSU = NDS_solution.T

            # If candidate is dominated by an archived solution
            if tmp_Ttot <= ca_Ttot and tmp_ECTot <= ca_ECtot and tmp_MSU <= ca_MSU:
               # Nothing to change
               return currentSol_candidateSol
            
            # If candidate dominates NDS_solution then reject the archived solution
            elif tmp_Ttot > ca_Ttot and tmp_ECTot > ca_ECtot and tmp_MSU > ca_MSU:
                dominated_solutions.append(id)

        if len(dominated_solutions) > 0:
            #TODO remove dominated solutions from archive
            #TODO add candidate solution to the archive and do not change current solution
            return currentSol_candidateSol

        # Candidate does not dominate any archived solution neither does any archived dominate it.
        else:
            #TODO If archive not full (refpop) : add candidate to archive
            # if archive full, choose between current_solution and candidate based on region density.
            return currentSol_candidateSol
            


# TEST

Initialisation

Constantes

In [ ]:
# Bandes passantes en Mb/s
BP_p : 1 # local
BP_q : 1 # inter-MEC
BP_r : 1 # MEC-cloud

BP_cumsum = np.cumsum(np.array([1/BP_p,1/BP_q,1/BP_r]))

# La performance unitaire d’une VM en Mb/s
P_u : 1 

# Energie
# Puissances électrique en W
r_v : 350 #puissance d'un serveur à vide
r_o : 75 #puissance d'une VM occupée
r_u : 45 #puissance d'une VM innocupée

Variables du test

In [ ]:
# 5 MEC Servers, 10 tâches, 3VM par MEC
M = 5
N = 10
V = np.array([3 for _ in range(M)])

# On attribue arbitrairement les N tâches au M MEC
MEC = np.array([int(n/2) for n in range(N)])
X = np.array([[1 if m==MEC[n] else 0 for n in range(N)] for m in range(M)])

# On initialise les tâches à 2 Mbits
G = np.array([2 for _ in range(N)])
# On initialise les resultats à 1 Mbit
RG = np.array([1 for _ in range(N)])

# On initialise les MEC_exe au MEC parents
MEC_exe = MEC.copy()

# Initially all tasks are done on their local MEC so j=0
J = np.zeros(N)

# On alloue une VM pour executer les tâches
U = np.array([1 for _ in range(N)])

# On compte les VM des tâches non-cloud, ici U_NC = U
U_NC =  U_NC_func(U, MEC_exe)

In [ ]:
C_0 = C_func(MEC_exe,U)
print(C_0)